In [1]:
import os
import pandas as pdsumma
import geopandas as gpd

In [2]:
path = r'C:\Users\clid1852\OneDrive - lanecouncilofgovernments\data\RTP\Webmaps'

In [3]:
data_2045 = pd.read_excel(os.path.join(path, 'kate_mixed-use_vectors_2020_2045_LCOG.xlsx'), 
                          skiprows = 1, usecols = 'F:I', nrows = 666)

In [4]:
colnames = ['transit_pk', 'transit_op', 'auto_pk', 'auto_op']

In [5]:
data_2045.columns = [fieldnm + '_45' for fieldnm in colnames]

In [6]:
data_2045.head()

,transit_pk_45,transit_op_45,auto_pk_45,auto_op_45
0,1111.685130,8.060288,121684.542084,122060.507068
1,0.000000,0.000000,5792.200921,5378.831521
2,31259.129235,32029.577063,116242.591222,122044.526664
3,0.000000,0.000000,67900.827949,74071.527568
4,5731.801513,6118.215998,83349.686810,89174.201837


In [7]:
for colnm in colnames:  
    if colnm == 'transit_pk':
        data = pd.read_csv(os.path.join(path, '2020_emp_within_30_mins_' + colnm + '.csv'), header=None)
        data.columns = ['taz', colnm + '_20']
    else:
        dt = pd.read_csv(os.path.join(path, '2020_emp_within_30_mins_' + colnm + '.csv'), header=None)
        data[colnm + '_20'] = dt.iloc[:,1]

In [8]:
data.head()

,taz,transit_pk_20,transit_op_20,auto_pk_20,auto_op_20
0,1,0.0,0.0,98838.647067,99339.546307
1,2,0.0,0.0,5392.756708,5400.144022
2,3,0.0,0.0,92648.396416,96215.162088
3,4,0.0,0.0,55439.150767,59145.997129
4,5,0.0,0.0,66474.475476,69981.741174


In [26]:
df = pd.concat([data, data_2045], axis=1)

In [23]:
df.head()

,taz,transit_pk_20,transit_op_20,auto_pk_20,auto_op_20,transit_pk_45,transit_op_45,auto_pk_45,auto_op_45
0,1,0.0,0.0,98838.647067,99339.546307,1111.685130,8.060288,121684.542084,122060.507068
1,2,0.0,0.0,5392.756708,5400.144022,0.000000,0.000000,5792.200921,5378.831521
2,3,0.0,0.0,92648.396416,96215.162088,31259.129235,32029.577063,116242.591222,122044.526664
3,4,0.0,0.0,55439.150767,59145.997129,0.000000,0.000000,67900.827949,74071.527568
4,5,0.0,0.0,66474.475476,69981.741174,5731.801513,6118.215998,83349.686810,89174.201837


In [27]:
df = df.apply(lambda x: x+0.5, axis=1).astype(int)

In [28]:
df.head()

,taz,transit_pk_20,transit_op_20,auto_pk_20,auto_op_20,transit_pk_45,transit_op_45,auto_pk_45,auto_op_45
0,1,0,0,98839,99340,1112,8,121685,122061
1,2,0,0,5393,5400,0,0,5792,5379
2,3,0,0,92648,96215,31259,32030,116243,122045
3,4,0,0,55439,59146,0,0,67901,74072
4,5,0,0,66474,69982,5732,6118,83350,89174


In [29]:
mtaz16 = gpd.read_file(r'V:\Data\Transportation\MTAZ16.shp')

In [18]:
mtaz16.head()

,taz,DIST10,DIST30,Shape_Leng,Shape_Area,geometry,transit_pk_20,transit_op_20,auto_pk_20,auto_op_20,transit_pk_45,transit_op_45,auto_pk_45,auto_op_45
0,584,9,28,20161.540478,1.988889e+07,"POLYGON ((4266795.748 856211.711, 4266865.691 ...",0,0,128528,128503,438,0,162508,162857
1,463,2,5,60626.352092,1.234526e+08,"POLYGON ((4247285.755 854987.644, 4247478.463 ...",0,0,124119,124786,3469,3585,154738,156935
2,581,9,28,21413.484541,1.937114e+07,"POLYGON ((4261648.531 856417.278, 4261576.736 ...",0,0,130007,129982,21994,21725,165306,165712
3,432,2,7,21763.737348,1.277206e+07,"POLYGON ((4237728.735 859170.524, 4237686.042 ...",0,0,96727,99347,22929,15628,118718,122865
4,585,9,28,16111.548256,1.392528e+07,"POLYGON ((4274245.976 855943.666, 4274263.337 ...",0,0,127388,127361,572,0,160438,160711


In [30]:
mtaz16 = mtaz16.rename(columns={'TAZ_NUM':'taz'})

In [31]:
mtaz16 = mtaz16.merge(df, on = 'taz')

In [32]:
mtaz16 = mtaz16.rename(columns={'transit_pk_20':'tran_pk_20', 
                                'transit_op_20':'tran_op_20',
                                'transit_pk_45':'tran_pk_45', 
                                'transit_op_45':'tran_op_45'})

In [33]:
mtaz16.to_file(r'T:\MPO\RTP\FY20 2045 Update\Data and Resources\Webmapping\Work_Accessibility\TOT30.shp')